# 🎯 Vertebrae Keypoint Estimation - Google Colab

**ECCV 2024: Bones Can't Be Triangles**

This notebook runs the complete training pipeline for vertebrae keypoint estimation.

---

## 📋 Before Running:

1. **Enable GPU**: `Runtime > Change runtime type > GPU > Save`
2. **Update Repository URL** in Cell 1 below
3. **Run cells in order** (don't skip cells)
4. **Training takes several hours** - keep tab active

---


## 1️⃣ Setup and Clone Repository

🔴 **UPDATE `REPO_URL` below with your GitHub repository URL**


In [ ]:
import os
import sys

# Check GPU
import torch
if torch.cuda.is_available():
    print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
else:
    print("❌ NO GPU! Please enable GPU:")
    print("   Runtime > Change runtime type > GPU > Save")

# 🔴 UPDATE THIS with your GitHub repository URL
REPO_URL = "https://github.com/YOUR_USERNAME/YOUR_REPO_NAME.git"

# Clone repository
repo_name = REPO_URL.split('/')[-1].replace('.git', '')

if not os.path.exists(repo_name):
    print(f"📥 Cloning repository...")
    !git clone {REPO_URL}
else:
    print(f"✅ Repository exists, pulling latest...")
    os.chdir(repo_name)
    !git pull
    os.chdir('..')

# Navigate to repository
os.chdir(repo_name)
print(f"✅ Working directory: {os.getcwd()}")

# Verify structure
if os.path.exists('codes'):
    print("✅ Project structure verified")
    !ls -la codes/
else:
    print("❌ 'codes' directory not found!")


## 2️⃣ Install Dependencies

This takes 5-10 minutes.


In [ ]:
print("📦 Installing dependencies...")

# Install PyTorch with CUDA
!pip install -q torch==1.12.0+cu113 torchvision==0.13.0+cu113 torchaudio==0.12.0+cu113 --extra-index-url https://download.pytorch.org/whl/cu113

# Install core packages
!pip install -q opencv-python==4.6.0.66 albumentations==1.2.1 PyYAML==6.0
!pip install -q tensorboard==2.12.0 timm==0.9.7 scikit-image==0.19.3
!pip install -q pytz==2022.1 tqdm==4.64.0 matplotlib==3.5.2 pandas==1.4.3 numpy==1.23.1

# Try full requirements
!pip install -q -r requirements.txt 2>&1 | grep -v ERROR || true

print("✅ Dependencies installed!")


## 3️⃣ Preprocess Data


In [ ]:
print("🔄 Preprocessing data...")

%cd codes/preprocess_data
!python preprocess_data.py
%cd ../..

print("✅ Data preprocessed!")


## 4️⃣ Train Main Keypoint Model

⏱️ This takes several hours (4-8 hours on T4 GPU)


In [ ]:
print("🚀 Training main keypoint model...\n")

%cd codes

# THIS IS THE MAIN TRAINING EXECUTION
!python -u main.py --seed 42 --config config_AASCE --save_test_prediction --subpixel_inference 15 --use_prev_heatmap_only_for_hint_index

%cd ..

print("\n✅ Main model training complete!")


## 5️⃣ Train AASCE Models (Detector + Corrector)

⏱️ This takes 1-2 hours


In [ ]:
print("🚀 Training AASCE models...\n")

%cd codes

# THIS TRAINS THE DETECTOR AND CORRECTOR
!python train_AASCE.py

%cd ..

print("\n✅ AASCE models training complete!")


## 6️⃣ Run Evaluation (Inference)


In [ ]:
print("📊 Running evaluation...\n")

%cd codes

# EVALUATION/INFERENCE
!python evaluate_AASCE.py

%cd ..

print("\n✅ Evaluation complete!")
print("\n🎉 ALL TRAINING AND EVALUATION COMPLETED SUCCESSFULLY!")


## 7️⃣ (Optional) Save Results to Google Drive


In [ ]:
from google.colab import drive
from datetime import datetime

# Mount Google Drive
drive.mount('/content/drive')

# Create timestamped results directory
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
result_dir = f'/content/drive/MyDrive/vertebrae_results_{timestamp}'

!mkdir -p {result_dir}

print("💾 Saving results to Google Drive...")

# Copy model files
!find . -name "*.pth" -type f -exec cp {} {result_dir}/ \; 2>/dev/null

# Copy save directories
!cp -r save_* {result_dir}/ 2>/dev/null || echo "No save_ folders"

# Copy logs
!cp -r codes/logs {result_dir}/ 2>/dev/null || echo "No logs"

print(f"\n✅ Results saved to: {result_dir}")
!ls -lh {result_dir}/
